In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import xarray as xr
from tqdm import tqdm

In [2]:
#mds_solar=xr.open_mfdataset('../../yearly_datadir/year*2019*.nc',preprocess=preprocess_solar,combine='nested', concat_dim='time',parallel=True,chunks={'x': 20, 'y': 20})
mds_wind = xr.open_mfdataset('/scratch/vishald/jrf2_monsoonlab/shiv/download_data/Wind_*.nc',combine = 'nested',concat_dim = 'time', parallel=True, chunks={'longitude': 200, 'latitude': 200})
u = mds_wind['u100']
v = mds_wind['v100']
speed = (u*u + v*v)**0.5
wpd_all = 0.5*1.2*(speed**3)
wpd_all

<xarray.DataArray (time: 350640, latitude: 353, longitude: 319)>
dask.array<mul, shape=(350640, 353, 319), dtype=float32, chunksize=(8784, 200, 200), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float32 65.55 65.65 65.75 65.85 ... 97.15 97.25 97.35
  * latitude   (latitude) float32 38.95 38.85 38.75 38.65 ... 3.95 3.85 3.75
  * time       (time) datetime64[ns] 1981-01-01 ... 2020-12-31T23:00:00

In [3]:
#mds_solar=xr.open_mfdataset('../../yearly_datadir/year*2019*.nc',preprocess=preprocess_solar,combine='nested', concat_dim='time',parallel=True,chunks={'x': 20, 'y': 20})
mds_solar = xr.open_mfdataset('/scratch/vishald/jrf2_monsoonlab/shiv/download_data/Solar_*.nc',combine = 'nested',concat_dim = 'time', parallel=True, chunks = {'latitude':200, 'longitude':200})
mds_solar = mds_solar['ssrd']
mds_solar = mds_solar/3600
day_threshold = 10
mds_solar = mds_solar.where( mds_solar >= day_threshold )

In [4]:
from dask.distributed import Client
client = Client(n_workers=16, threads_per_worker=8, memory_limit='128GB')
#client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 128,Total memory: 1.86 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43381,Workers: 16
Dashboard: http://127.0.0.1:8787/status,Total threads: 128
Started: Just now,Total memory: 1.86 TiB
Comm: tcp://127.0.0.1:42009,Total threads: 8
Dashboard: http://127.0.0.1:37881/status,Memory: 119.21 GiB
Nanny: tcp://127.0.0.1:36155,


In [5]:
import gc
client.run(gc.collect)

{'tcp://127.0.0.1:35479': 68,
 'tcp://127.0.0.1:35973': 68,
 'tcp://127.0.0.1:36023': 68,
 'tcp://127.0.0.1:36053': 68,
 'tcp://127.0.0.1:36133': 68,
 'tcp://127.0.0.1:37157': 68,
 'tcp://127.0.0.1:37361': 68,
 'tcp://127.0.0.1:37893': 68,
 'tcp://127.0.0.1:38395': 68,
 'tcp://127.0.0.1:38773': 68,
 'tcp://127.0.0.1:40095': 68,
 'tcp://127.0.0.1:42009': 68,
 'tcp://127.0.0.1:42063': 68,
 'tcp://127.0.0.1:42105': 68,
 'tcp://127.0.0.1:45359': 68,
 'tcp://127.0.0.1:46429': 68}

In [6]:
import ctypes
def trim_memory() -> int:
    libc = ctypes.CDLL("libc.so.6")
    return libc.malloc_trim(0)
client.run(trim_memory)

{'tcp://127.0.0.1:35479': 1,
 'tcp://127.0.0.1:35973': 1,
 'tcp://127.0.0.1:36023': 1,
 'tcp://127.0.0.1:36053': 1,
 'tcp://127.0.0.1:36133': 1,
 'tcp://127.0.0.1:37157': 1,
 'tcp://127.0.0.1:37361': 1,
 'tcp://127.0.0.1:37893': 1,
 'tcp://127.0.0.1:38395': 1,
 'tcp://127.0.0.1:38773': 1,
 'tcp://127.0.0.1:40095': 1,
 'tcp://127.0.0.1:42009': 1,
 'tcp://127.0.0.1:42063': 1,
 'tcp://127.0.0.1:42105': 1,
 'tcp://127.0.0.1:45359': 1,
 'tcp://127.0.0.1:46429': 1}

In [4]:

    mds_solar_all = mds_solar
    from tqdm import tqdm
    for start_month, end_month in tqdm([[6,9],[10,11],[12,2],[3,5]]):

        if start_month == 12:
            mds_solar = mds_solar_all.sel(time=np.logical_or(mds_solar_all.time.dt.month >= start_month, mds_solar_all.time.dt.month <= end_month))
        else:
            mds_solar = mds_solar_all.sel(time=np.logical_and(mds_solar_all.time.dt.month >= start_month, mds_solar_all.time.dt.month <= end_month))

        test_solar_mean = (mds_solar[0:4]).mean('time',skipna=True).compute() 
        print('Test Solar Mean computed successfully. Proceeding to Compute Entire Data Mean')
        
        solar_mean = (mds_solar).mean('time',skipna=True).compute() 
        solar_mean.to_netcdf(f'/scratch/vishald/jrf2_monsoonlab/shiv/datastats/{start_month}-{end_month} Solar Mean.nc')
        print('Entire Solar Mean computed successfully. Proceeding to Compute Entire Data Median')

        solar_median = (mds_solar).median('time',skipna=True).compute() 
        solar_median.to_netcdf(f'/scratch/vishald/jrf2_monsoonlab/shiv/datastats/{start_month}-{end_month} Solar Median.nc')
        print('Entire Wind Median computed successfully. Proceeding to Compute Entire Data IQR')

        #from scipy.stats import iqr
        #solar_iqr = iqr(mds_solar,0,nan_policy = 'omit')
        #np.save(f'Data Stats/{start_month}-{end_month} Solar IQR.npy',solar_iqr)

        solar_rcov = ((abs(mds_solar - solar_median)).median('time',skipna=True).compute())/solar_median
        solar_rcov.to_netcdf(f'/scratch/vishald/jrf2_monsoonlab/shiv/datastats/{start_month}-{end_month} Solar RCoV.nc')

        solar_th = 170
        wind_th = 240
        day_time_hours = mds_solar.count(dim='time').load()
        solar_avail = mds_solar.where(mds_solar > solar_th).count(dim='time').compute()
        pct_solar_avail = (solar_avail/day_time_hours)*100
        pct_solar_avail.to_netcdf(f'/scratch/vishald/jrf2_monsoonlab/shiv/datastats/{start_month}-{end_month} Solar Availability.nc')
        print('Done')



  0%|          | 0/4 [00:00<?, ?it/s]/opt/apps/anaconda3-2023/lib/python3.10/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


Test Solar Mean computed successfully. Proceeding to Compute Entire Data Mean
Entire Solar Mean computed successfully. Proceeding to Compute Entire Data Median
Entire Wind Median computed successfully. Proceeding to Compute Entire Data IQR


 25%|██▌       | 1/4 [1:01:09<3:03:27, 3669.28s/it]

Done
Test Solar Mean computed successfully. Proceeding to Compute Entire Data Mean
Entire Solar Mean computed successfully. Proceeding to Compute Entire Data Median
Entire Wind Median computed successfully. Proceeding to Compute Entire Data IQR


 50%|█████     | 2/4 [2:00:52<2:00:37, 3618.67s/it]

...

Done


/opt/apps/anaconda3-2023/lib/python3.10/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


Test Solar Mean computed successfully. Proceeding to Compute Entire Data Mean
Entire Solar Mean computed successfully. Proceeding to Compute Entire Data Median
Entire Wind Median computed successfully. Proceeding to Compute Entire Data IQR


100%|██████████| 4/4 [4:01:26<00:00, 3621.72s/it]  

Done


In [7]:
    mds_wind_all= wpd_all
    for start_month, end_month in tqdm([[3,5]]):

        if start_month == 12:
            mds_wind = mds_wind_all.sel(time=np.logical_or(mds_wind_all.time.dt.month >= start_month, mds_wind_all.time.dt.month <= end_month))
        else:
            mds_wind = mds_wind_all.sel(time=np.logical_and(mds_wind_all.time.dt.month >= start_month, mds_wind_all.time.dt.month <= end_month))

        test_wind_mean = (mds_wind[0:4]).mean('time',skipna=True).compute() 
        print('Test wind Mean computed successfully. Proceeding to Compute Entire Data Mean')
        wind_median = (mds_wind_all).median('time',skipna=True).compute() 

        wind_rcov = ((abs(mds_wind - wind_median)).median('time',skipna=True).compute())/wind_median
        wind_rcov.to_netcdf(f'/scratch/vishald/jrf2_monsoonlab/shiv/datastats/{start_month}-{end_month} Wind RCoV.nc')

        '''solar_th = 170
        wind_th = 240
        day_time_hours = mds_wind.count(dim='time').load()
        wind_avail = mds_wind.where(mds_wind > wind_th).count(dim='time').compute()
        pct_wind_avail = (wind_avail/day_time_hours)*100
        pct_wind_avail.to_netcdf(f'/scratch/vishald/jrf2_monsoonlab/shiv/datastats/{start_month}-{end_month} Wind Availability.nc')'''
        print('Done')

  0%|          | 0/1 [00:00<?, ?it/s]/opt/apps/anaconda3-2023/lib/python3.10/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


Test wind Mean computed successfully. Proceeding to Compute Entire Data Mean


100%|██████████| 1/1 [58:41<00:00, 3521.45s/it]

Done


In [8]:
    mds_wind_all = wpd_all
    for start_month, end_month in tqdm([[12,2],[3,5]]):

        if start_month == 12:
            mds_wind = mds_wind_all.sel(time=np.logical_or(mds_wind_all.time.dt.month >= start_month, mds_wind_all.time.dt.month <= end_month))
        else:
            mds_wind = mds_wind_all.sel(time=np.logical_and(mds_wind_all.time.dt.month >= start_month, mds_wind_all.time.dt.month <= end_month))

        test_wind_mean = (mds_wind[0:4]).mean('time',skipna=True).compute() 
        print('Test wind Mean computed successfully. Proceeding to Compute Entire Data Mean')

        solar_th = 170
        wind_th = 240
        day_time_hours = mds_wind.count(dim='time').load()
        wind_avail = mds_wind.where(mds_wind > wind_th).count(dim='time').compute()
        pct_wind_avail = (wind_avail/day_time_hours)*100
        pct_wind_avail.to_netcdf(f'/scratch/vishald/jrf2_monsoonlab/shiv/datastats/{start_month}-{end_month} Wind Availability.nc')
        print('Done')

  0%|          | 0/2 [00:00<?, ?it/s]/opt/apps/anaconda3-2023/lib/python3.10/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


Test wind Mean computed successfully. Proceeding to Compute Entire Data Mean


 50%|█████     | 1/2 [1:09:06<1:09:06, 4146.93s/it]

Done


/opt/apps/anaconda3-2023/lib/python3.10/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


Test wind Mean computed successfully. Proceeding to Compute Entire Data Mean


100%|██████████| 2/2 [2:26:38<00:00, 4399.40s/it]  

Done
